In [30]:
from __future__ import print_function
import keras,sklearn
# suppress tensorflow compilation warnings
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import tensorflow.compat.v1 as tf
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import numpy as np
seed=0
np.random.seed(seed) # fix random seed
tf.compat.v1.set_random_seed(seed)
import matplotlib.pyplot as plt
import cv2
import pickle
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from tensorflow.keras import datasets, layers, models
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
import requests
import io
from PIL import Image, ImageTk
import tkinter as tk
from tkinter.filedialog import asksaveasfilename
from tkinter.filedialog import askopenfilename
from numpy import asarray
import time
import tkinter as tk
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt

In [31]:
#load model from server
url_main = 'https://physics.bu.edu/~warsh/ML_QD/'
modelfile = 'model_200000samps.h5'

response = requests.get(url_main + modelfile)
response.raise_for_status()

model_200k = load_model(io.BytesIO(response.content))

model=model_200k

In [10]:
#load model from local dir
mod_dir = 'C:/Users/Warsh/ML_Notebooks_edited/mlqdproj/models/'
modelfile = 'l18e-06l20b512oAdaml16.h5'

model = load_model(mod_dir + modelfile)

In [32]:
#upload image for classification cell
#image directory
userdir='C:/Users/Warsh/ML_Notebooks_edited/mlqdproj/drawings/'

#upload and format file, then send to classifier
def openFile():
    global imagepath
    imagepath = askopenfilename(title="Select User Settings", initialdir=userdir)
    image=Image.open(imagepath)
    if image.size[0]>image.size[1]:
        image=image.crop((image.size[0]/2-image.size[1]/2,0,image.size[0]/2+image.size[1]/2,image.size[1]))
    elif image.size[0]>image.size[1]:
        image=image.crop((0,image.size[1]/2-image.size[0]/2,image.size[0],image.size[1]/2+image.size[0]/2))
    else:
        image=image
    basewidth = 512
    wpercent = (basewidth/float(image.size[0]))
    hsize = int((float(image.size[1])*float(wpercent)))
    image = image.resize((basewidth,hsize), Image.ANTIALIAS)
    img = asarray(image)
    rgb_weights = [0.2989, 0.5870, 0.1141]
    gs_img1 = np.dot(img[...,:3], rgb_weights)
    gs_img = abs(255-gs_img1)/255
    lowres = cv2.resize(gs_img, dsize=(28, 28), interpolation=cv2.INTER_LINEAR)
    root.imagine = ImageTk.PhotoImage(Image.fromarray(gs_img1))
    change_pic()
    classify(lowres)

#draw pic and send to classifier
def draworama():
    global drawing,mode
    drawing = False # true if mouse is pressed
    mode = False # if True, draw rectangle. Press 'm' to toggle to curve
    ix,iy = -1,-1
    brushsize=10
    ersize=20
    # mouse callback function
    def draw_circle(event,x,y,flags,param):
        global ix,iy,drawing,mode

        #this is eraser
        if event == cv2.EVENT_RBUTTONDOWN:
            mode = not(mode)
            ix,iy = x,y

        #and this is for drawing
        if event == cv2.EVENT_LBUTTONDOWN:
            drawing = True
            ix,iy = x,y

        elif event == cv2.EVENT_MOUSEMOVE:
            if drawing == True:
                if mode == True:
                    cv2.circle(img,(x,y),ersize,(255,255,255),-1)
                else:
                    cv2.circle(img,(x,y),brushsize,(0,0,0),-1)

        elif event == cv2.EVENT_LBUTTONUP:
            drawing = False
            if mode == True:
                cv2.circle(img,(x,y),ersize,(255,255,255),-1)
            else:
                cv2.circle(img,(x,y),brushsize,(0,0,0),-1)

    #create image and a window and bind the function to window
    img = np.zeros((512,512,3), np.uint8)
    cv2.namedWindow('draw something!')
    cv2.circle(img,(255,255),400,(255,255,255),-1)
    cv2.setMouseCallback('draw something!',draw_circle)
    while(1):
        cv2.imshow('draw something!',img)
        if cv2.waitKey(20) & 0xFF == 27:
            break

    #convert to usable
    rgb_weights = [0.2989, 0.5870, 0.1141]
    gs_img1 = np.dot(img[...,:3], rgb_weights)
    gs_img = abs(255-gs_img1)/255
    lowres = cv2.resize(gs_img, dsize=(28, 28), interpolation=cv2.INTER_LINEAR)
    root.imagine = ImageTk.PhotoImage(Image.fromarray(img))
    change_pic()
    classify(lowres)
    #close windows
    cv2.destroyAllWindows()

#quit
def quitorama():
    root.destroy()

#change the displayed pic
def change_pic():
    vlabel.configure(image=root.imagine)

#classify
def classify(img):
    lowres = cv2.resize(img, dsize=(28, 28), interpolation=cv2.INTER_LINEAR)
    lowres_rshp=lowres.reshape(1,28,28,1)
    #predict with the trained CNN
    yokay=['airplane', 'monalisa', 'dragon', 'giraffe', 'axe', 'banana', 'eiffeltower', 'snail', 'windmill', 'snowman']
    cat=model.predict(lowres_rshp).argmax(1)
    cert=100*model.predict(lowres_rshp).max()
    lbl.config(text='CNN classified as ' + yokay[cat[0]] + ' with %.2f%% certainty'%cert)

In [33]:
root = tk.Tk()

url_main = 'https://physics.bu.edu/~warsh/ML_QD/'
response = requests.get(url_main + 'introslide.png')
response.raise_for_status()
photo = io.BytesIO(response.content)
#photo = 'C:/Users/Warsh/ML_Notebooks_edited/mlqdproj/drawings/introslide.png'
root.photo = ImageTk.PhotoImage(Image.open(photo))

lbl = tk.Label(root, text="classify your drawing or image")
lbl.pack()

vlabel=tk.Label(root,image=root.photo)
vlabel.pack()

b2=tk.Button(root,text="upload",height = 2, 
          width = 20,command=openFile)
b2.pack()
b3=tk.Button(root,text="draw",height = 2, 
          width = 20,command=draworama)
b3.pack()
q=tk.Button(root,text="quit",height = 2, 
          width = 20,command=quitorama)
q.pack()

root.mainloop()